# Traffic Sign Recognition Model

**Dataset**: [GTSRB - German Traffic Sign Recognition Benchmark](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign)

**Goal**: To create as accurate model as possible with output model weighing less than 100MB so it's usabel on Raspberry Pi with compressing in TensorFlow Lite

**outcome: 95.26% of accuracy on a test test and 77MB model**

<hr>
Here I've decided to use batch size of 16 with pictures of 32x32 pixels. 
I tried using bigger/smaller pictures with different batch sizes. While lowering batch size and increasing image size didn't affect the accuracy that much I've decided to stick to those values to decrease training time significantly.

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Rescaling, Conv2D, Dense, MaxPooling2D, Dropout, Flatten

PATH = '/kaggle/input/gtsrb-german-traffic-sign/'
BATCH_SIZE = 16
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32

After imports I've exctracted tabular data from Train.csv and extracted file paths and labels to create traning dataset. 
I looped over entire dataset and processed every image (basically loading and resizing every picture), then I shuffled dataset and created batches.
At the end dataset was split in two for train and validation datasets with ratio 8:2.

In [2]:
# Load file paths and labels
data = pd.read_csv(PATH + 'Train.csv')
file_names = data['Path'].values
file_paths = tf.constant([os.path.join(PATH, fn) for fn in file_names])

labels = data['ClassId'].values
labels = tf.constant(labels)
num_classes = len(np.unique(labels))

# Convert to TensorFlow Dataset and normalize images
def process_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMAGE_WIDTH, IMAGE_HEIGHT])
    return img, label

dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
dataset = dataset.map(process_image)
dataset = dataset.shuffle(buffer_size=len(file_paths), seed=42)
dataset = dataset.batch(BATCH_SIZE)
train, valid = tf.keras.utils.split_dataset(dataset, left_size=0.8)

Now I am loading test set, for that we need to extract labels from Test.csv and then we provide image directory to image_dataset_from_directory function. <br><br>
<i>It would have been great to use the same function for prodiving test and validation sets, but TensorFlow couldn't infer labels correctly even while the folder structure seemed correct. So I've decided to load dataset manually (almost) with from_tensor_slices.</i>

In [3]:
test_labels = list(pd.read_csv(PATH + 'Test.csv', usecols=['ClassId']).to_numpy().flatten())
test = tf.keras.preprocessing.image_dataset_from_directory(
    PATH + 'Test',
    labels=test_labels,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    interpolation='bilinear',
    crop_to_aspect_ratio=True,
    verbose=True
)

Found 12630 files belonging to 43 classes.


Here I am preprocessing the data with optimizing training time by prefetching data into buffers. Also I am augmenting images in the training set.

In [4]:
data_augmentation = Sequential([
    tf.keras.layers.RandomRotation(0.04),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    tf.keras.layers.RandomBrightness(factor=0.15),
    tf.keras.layers.GaussianNoise(0.12),
])

AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(train, augment=True)
val_ds = prepare(valid)
test_ds = prepare(test)

Displaying batch of images.

In [ ]:
plt.figure(figsize=(30, 30))
for x, y in train_ds.take(1):
  for i in range(BATCH_SIZE):
    ax = plt.subplot(4, 4, i + 1)
    # Get the i-th label
    label = y[i]

    # Convert to displayable uint8 image
    display_image = (x[i]).numpy().astype("uint8")

    # Show image
    plt.imshow(display_image)
    plt.title(label.numpy())

So here is the heart of my model, I've decided on certain kernel_sizes based on Yan Han and Erdal Oruklu paper:
[Traffic sign recognition based on the NVIDIA Jetson TX1 embedded system using convolutional neural networks](https://www.researchgate.net/publication/320606581_Traffic_sign_recognition_based_on_the_NVIDIA_Jetson_TX1_embedded_system_using_convolutional_neural_networks)
<hr>
To avoid a dead relu problem, I've used LeakyRelu activation for convolutions, while sticking to basic relu in Dense layers. First tries were based on around 25M parameters with higher filters and conv layers number. Final version uses around 6.4M parameters and is much lighter, I probably could have shrinked it even more, but decrease in weight to 77MB satisfied my needs without hurting accuracy and models ability to generalize unseen data.

In [5]:
def create_model():
    model = Sequential([
        tf.keras.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
        # Data normalization
        Rescaling(1./255),
        # Convolutional layers
        Conv2D(50, kernel_size=(9,9), strides=1, padding="same"),
        tf.keras.layers.LeakyReLU(),
        MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding="same"),
        Conv2D(70, kernel_size=(7,7), strides=1, padding="same"),
        tf.keras.layers.LeakyReLU(),
        MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding="same"),
        Conv2D(100, kernel_size=(3,3), strides=1, padding="same"),
        tf.keras.layers.LeakyReLU(),
        # Dense layers
        Flatten(),
        Dense(60, activation='relu'),
        Dropout(0.2),
        Dense(60, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    return model

Here I am specyfing my early stopper so I dont have to worry about number of epochs ran to train the model. Instead I watch the accuracy from validation set to rise at least by 0.5% within 20 epochs.

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=20, restore_best_weights=True)

In [6]:
model = create_model()
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 32, 50)          │          12,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 32, 32, 50)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 50)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 70)          │         171,570 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 32, 32, 70)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 70)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 100)         │          63,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32, 32, 100)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 102400)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 60)                  │       6,144,060 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 43)                  │           2,623 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,397,213 (24.40 MB)

 Trainable params: 6,397,213 (24.40 MB)

 Non-trainable params: 0 (0.00 B)

And now we start to train the model, number of epochs doesnt really matter as long as its a high number, early stopping is going to take care of ending the training when its suitable to do so. All hyperparameters in here were chosen based on some trial and errors.

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0003),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=1000, verbose=True, callbacks=[early_stop],
                    validation_data=val_ds, shuffle=True)

Plotting our efficiency on the training and validation data.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(100)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Saving keras model.

In [ ]:
model.save('tsr_maxpool_v3.keras')

Evaluating model on the test set.

In [ ]:
results = model.evaluate(test, batch_size=BATCH_SIZE)
print("test loss, test acc:", results)

Visualizing predicted and original labels for batches of test data.

In [ ]:
for images, labels in test.take(1):  # This grabs a batch
    plt.figure(figsize=(20, 20))
    for i in range(BATCH_SIZE):  # Display 16 images
        ax = plt.subplot(4, 4, i + 1)

        image = images[i]
        label = labels[i]

        predict_image = tf.expand_dims(image, axis=0)
        prediction = model.predict(predict_image, verbose=0)
        y_hat = np.argmax(prediction, axis=1)[0]

        plt.imshow(image.numpy().astype("uint8"))
        plt.title(f'Original: {label.numpy()}, Predicted: {y_hat}')
        plt.axis("off")